In [9]:
path = r"C:\Users\shank\OneDrive\Desktop\New folder\archive\mitbih_database"
filenames = next(os.walk(path))[2]
records=list()
annotations=list()
filenames.sort()

In [10]:
for f in filenames:
    filename, file_extension = os.path.splitext(f)

    if(file_extension=='.csv'):
        records.append(path+'/'+filename+file_extension)
    else:
        annotations.append(path+'/'+filename+file_extension)

records[0]

'C:\\Users\\shank\\OneDrive\\Desktop\\New folder\\archive\\mitbih_database/100.csv'

In [11]:
X=list()
y=list()
window_size=180
for r in range(0,len(records)):
    signals = []

    with open(records[r], 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|') 
        row_index = -1
        for row in spamreader:
            if(row_index >= 0):
                signals.insert(row_index, int(row[1]))
            row_index += 1

        
    signals = denoise(signals)
    #signals = signals.reshape(-1,1)
    signals = stats.zscore(signals)

    with open(annotations[r], 'r') as fileID:
        data = fileID.readlines() 
        beat = list()

        for d in range(1, len(data)): 
            splitted = data[d].split(' ') 
            splitted = filter(None, splitted)
            next(splitted) 
            pos = int(next(splitted)) 
            arrhythmia_type = next(splitted) 
            
            if(arrhythmia_type in classes):
                arrhythmia_index = classes.index(arrhythmia_type)
                count_classes[arrhythmia_index] += 1
                if(window_size <= pos and pos < (len(signals) - window_size)):  
                    beat = signals[pos-window_size:pos+window_size]   

                    X.append(beat)
                    y.append(arrhythmia_index)

# data shape
print(np.shape(X), np.shape(y))

(100012, 360) (100012,)


In [ ]:
import pywt
import numpy as np
import csv
from scipy import stats
import matplotlib.pyplot as plt
from PIL import Image
import os
from sklearn.utils import resample

def denoise(signal):
    return signal  # Replace with actual implementation

# Function to save scalogram images
def save_image(scalogram, label, index):
    image_size = (227, 227)  # Target size for AlexNet
    plt.figure(figsize=(image_size[0] / 100, image_size[1] / 100), dpi=100)
    plt.imshow(scalogram, aspect='auto', cmap='jet')
    plt.axis('off')
    plt.tight_layout(pad=0)
    image_path = os.path.join(image_dir, f'scalogram_{label}_{index}.png')
    plt.savefig(image_path, bbox_inches='tight', pad_inches=0)
    plt.close()

# Initialization
X = []
y = []
window_size = 180
scales = np.arange(1, 64)  # Adjust scale range as needed

# Directories and class info
image_dir = 'images'
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

classes = ['N', 'L', 'R', 'A', 'V']  # Example class names
count_classes = [0] * len(classes)
target_samples_per_class = 2500

# Load and preprocess the data
for r in range(len(records)):
    signals = []
    with open(records[r], 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row_index, row in enumerate(spamreader):
            if row_index > 0:
                signals.append(int(row[1]))

    signals = denoise(signals)
    signals = stats.zscore(signals)

    with open(annotations[r], 'r') as fileID:
        data = fileID.readlines()
        for d in range(1, len(data)):
            splitted = data[d].split()
            pos = int(splitted[1])
            arrhythmia_type = splitted[2]

            if arrhythmia_type in classes:
                arrhythmia_index = classes.index(arrhythmia_type)
                if window_size <= pos < (len(signals) - window_size):
                    beat = signals[pos - window_size:pos + window_size]
                    X.append(beat)
                    y.append(arrhythmia_index)

from sklearn.utils import resample

# Initialize lists to store resampled data
resampled_X = []
resampled_y = []

# Define the target number of samples per class
target_samples_per_class = {
    0: 10000,
    1: 7000,
    2: 7000,
    3: 5000,
    4: 7000
}

# Iterate over each class index and perform resampling
for class_index in range(len(classes)):
    class_samples = [X[i] for i in range(len(X)) if y[i] == class_index]
    target_samples = target_samples_per_class[class_index]
    
    if class_index == 0:
        random_state = 42
    elif class_index == 1:
        random_state = 122
    elif class_index == 2:
        random_state = 123
    elif class_index == 3 or class_index == 4:
        random_state = 125
    
    # Perform resampling
    resampled_class_samples = resample(class_samples, n_samples=target_samples, random_state=random_state)
    
    # Extend resampled data to lists
    resampled_X.extend(resampled_class_samples)
    resampled_y.extend([class_index] * target_samples)


# Convert to scalogram and save images
batch_index = 0
for i in range(len(resampled_X)):
    beat = resampled_X[i]
    label = resampled_y[i]
    coefficients, _ = pywt.cwt(beat, scales, 'morl')
    scalogram = np.abs(coefficients)
    save_image(scalogram, label, batch_index)
    batch_index += 1

print(f"Saved {batch_index} scalogram images in the '{image_dir}' directory.")


In [16]:
#loding the images
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the absolute path to the image directory
image_dir = os.path.abspath('images')

# Verify if the directory exists and contains images for each class
if not os.path.exists(image_dir):
    print(f"Directory {image_dir} does not exist.")
else:
    classes = ['N', 'L', 'R', 'A', 'V']  # List of class names
    for class_name in classes:
        class_path = os.path.join(image_dir, class_name)
        if not os.path.exists(class_path):
            print(f"No images found for class {class_name}.")
        else:
            class_images = [f for f in os.listdir(class_path) if f.startswith('scalogram')]
            print(f"Found {len(class_images)} images for class {class_name}.")

# Define image dimensions
img_height, img_width = 227, 227

# Create an ImageDataGenerator for preprocessing
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

# Load training data
train_data = datagen.flow_from_directory(
    image_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Load validation data
validation_data = datagen.flow_from_directory(
    image_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

# Print summary
print("Summary:")
print(f"Training samples: {train_data.samples}")
print(f"Validation samples: {validation_data.samples}")


Found 10000 images for class N.
Found 7000 images for class L.
Found 7000 images for class R.
Found 5000 images for class A.
Found 7000 images for class V.
Found 28800 images belonging to 5 classes.
Found 7200 images belonging to 5 classes.
Summary:
Training samples: 28800
Validation samples: 7200


In [17]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Define directory containing all images
data_dir = r"D:\6th SEM\mini project\images"  

# Define image size and batch size
img_height, img_width = 227, 227
batch_size = 64

# Create ImageDataGenerator instance with rescaling
datagen = ImageDataGenerator(rescale=1.0/255)

# Create data generator
generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True  # Shuffle the data
)

# Calculate number of training and validation samples
total_samples = generator.samples
train_samples = int(0.8 * total_samples)  # 80% for training
val_samples = total_samples - train_samples  # 20% for validation

# Split the generator into training and validation sets
train_generator = tf.data.Dataset.from_generator(
    lambda: generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, img_height, img_width, 3], [None, 5])  
).take(train_samples).batch(batch_size)

validation_generator = tf.data.Dataset.from_generator(
    lambda: generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, img_height, img_width, 3], [None, 5])  
).skip(train_samples).batch(batch_size)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\6th SEM\\mini project\\images'

In [19]:
#trainingq
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Define directory containing all images
data_dir = r"C:\Users\shank\OneDrive\Desktop\New folder\images"   # Replace with your data directory containing subdirectories for each class

# Define image size and batch size
img_height, img_width = 227, 227
batch_size = 64

# Create ImageDataGenerator instance with rescaling
datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)  # Validation split if no separate validation directory

# Create data generator for training
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',  # Specify 'training' subset for training data
    shuffle=True  # Shuffle the data
)

# Create data generator for validation
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',  # Specify 'validation' subset for validation data
    shuffle=False  # No need to shuffle validation data
)

# Define the AlexNet model
def create_alexnet_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(96, (11, 11), strides=4, activation='relu', input_shape=(227, 227, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2),
        tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2),
        tf.keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(5, activation='softmax')  # 5 classes
    ])
    return model

# Create and compile the model
model = create_alexnet_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the weights of the best epoch
)

# Train the model with early stopping
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[early_stopping]  # Add the early stopping callback
)

print(f"Training finished. Best weights restored.")


Found 28800 images belonging to 5 classes.
Found 7200 images belonging to 5 classes.
Epoch 1/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 1815s 4s/step - accuracy: 0.5922 - loss: 1.3287 - val_accuracy: 0.9301 - val_loss: 0.2278
Epoch 2/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9399 - loss: 0.1890

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf

def create_alexnet_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(96, (11, 11), strides=4, activation='relu', input_shape=(227, 227, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2),
        tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2),
        tf.keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(5, activation='softmax')  # 5 classes
    ])
    model.summary()
    return model
alexnet_model = create_alexnet_model()


In [ ]:
import matplotlib.pyplot as plt

# Plot training history: model loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Plot training history: model accuracy
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


In [ ]:
# Evaluate the model on the validation set
validation_loss, validation_accuracy = model.evaluate(validation_data)
print(f"Validation Loss: {validation_loss}")
print(f"Validation Accuracy: {validation_accuracy}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming you want to test on a random subset of validation data
num_samples_per_class = 2  # Number of samples per class to test
classes = ['N', 'L', 'R', 'A', 'V']  # Class names

# Prepare lists to store test data and labels
test_data = []
test_labels = []

# Track number of samples per class
samples_count = {class_name: 0 for class_name in classes}

# Loop to accumulate batches from validation_generator
while any(count < num_samples_per_class for count in samples_count.values()):
    batch_data, batch_labels = next(validation_generator)
    for i in range(len(batch_labels)):
        class_label = np.argmax(batch_labels[i])
        if samples_count[classes[class_label]] < num_samples_per_class:
            test_data.append(batch_data[i])
            test_labels.append(batch_labels[i])
            samples_count[classes[class_label]] += 1

# Convert lists to numpy arrays
test_data = np.array(test_data)
test_labels = np.array(test_labels)

# Evaluate the model on the test data
print("Evaluating model on test data...")
test_loss, test_accuracy = model.evaluate(test_data, test_labels, batch_size=batch_size)

print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

# Make predictions on the test data
print("Making predictions on test data...")
predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

# Calculate number of rows and columns for subplots
num_classes = len(classes)
num_rows = num_samples_per_class
num_cols = num_classes

# Plot images with predictions
plt.figure(figsize=(15, 10))
for i in range(num_rows * num_cols):
    plt.subplot(num_rows, num_cols, i + 1)
    plt.imshow(test_data[i])
    plt.title(f"Predicted: {classes[predicted_classes[i]]}\nActual: {classes[np.argmax(test_labels[i])]}")
    plt.axis('off')

plt.tight_layout()
plt.show()

# Optionally, print predictions if needed
# for i in range(len(test_data)):
#     print(f"Sample {i+1}: Predicted class: {classes[predicted_classes[i]]}, Actual class: {classes[np.argmax(test_labels[i])]}")
